In [1]:
import cobra
import cobrakbase
kbase = cobrakbase.KBaseAPI()

cobrakbase 0.2.8


In [4]:
model_object = kbase.get_object('GCF_000009725.1.mdl.gf', 87388)

In [9]:
model_object['biomasses'][0]

{'biomasscompounds': [{'coefficient': -41.257,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'},
  {'coefficient': -0.00719527989638797,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00201_c0'},
  {'coefficient': -0.00719527989638797,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00006_c0'},
  {'coefficient': -0.186841915485094,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00161_c0'},
  {'coefficient': -0.0282246669459237,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00062_c0'},
  {'coefficient': -0.00719527989638797,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd10516_c0'},
  {'coefficient': 45.354000686262,
   'edits': {},
   'gapfill_data': {},
   'modelcompound_ref': '~/modelcompounds/id/cpd00001_c0'},
  {'coefficient': -0.0071952

In [11]:
from cobrakbase.core.kbasefba.fbamodel_biomass import Biomass
rxn = Biomass.from_json(model_object['biomasses'][0])

In [12]:
rxn

Reaction identifier,bio1
Name,GramPositiveBiomass auto biomass
Memory address,0x7f96045f1d00
Stoichiometry,--> -->
GPR,
Lower bound,0
Upper bound,1000


In [2]:
model_cobra = cobra.io.read_sbml_model('../../../../data/sbml/ec_core/e_coli_core_test.xml')

In [3]:
genome = kbase.get_from_ws('78503/2948/1')

In [4]:
model_cobra.compartments

{'e': 'extracellular space', 'c': 'cytosol'}

In [8]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
converter = CobraModelConverter(model_cobra, None, None)


In [9]:
a, b = converter.build()

In [14]:
for r in b.reactions:
    #print(r.gene_reaction_rule)
    r

In [10]:
kbase.save_object('test_model', 12218, 'KBaseFBA.FBAModel', b)

In [11]:
from cobrakbase.core.kbasefba.fbamodel_metabolite import ModelCompound
from cobrakbase.core.kbasefba.fbamodel_reaction import ModelReaction
from cobrakbase.core.kbasefba.fbamodel_biomass import Biomass
model_compartments = []
for cmp_id in converter.model.compartments:
    # TODO split cmp_id to extract index and compartment_ref no index assume 0
    model_compartments.append({
        'compartmentIndex': 0,
        'id': cmp_id,
        'label': converter.model.compartments[cmp_id],
        'compartment_ref': '~/template/compartments/id/' + cmp_id[0],
        'pH': 7,
        'potential': 0
    })

model_compounds = {}
for m in converter.model.metabolites:
    model_compound = ModelCompound(m.id, m.formula, m.name, m.charge, m.compartment)
    if model_compound.id not in model_compounds:
        model_compounds[model_compound.id] = model_compound
    else:
        logger.warning(f"duplicate metabolite {model_compound.id}")

model_reactions = []
for r in converter.metabolic_reactions:
    model_reaction = ModelReaction(r.id, r.name, r.subsystem, r.lower_bound, r.upper_bound,
                                   imported_gpr=r.gene_reaction_rule)
    model_reaction.gene_reaction_rule = r.gene_reaction_rule
    model_reaction.add_metabolites(dict([(model_compounds[c.id], v) for (c, v) in r.metabolites.items()]))
    model_reactions.append(model_reaction)


In [24]:
r.model

In [12]:
for r in model_reactions:
    r._to_json()

In [12]:
r.model_reaction_proteins

In [5]:
converter.build()

TypeError: 'NoneType' object is not iterable

In [172]:
model_cobra.annotation['modelseed'] = 'hello'
model_cobra.annotation['modelseeds'] = ['one', 'two', 'three']
model_cobra.notes = {
    'note1': ['one', 'two', 'three'],
    'note2': 'hello'
}
model_cobra.genes[0].annotation['modelseed'] = 'hello'
model_cobra.genes[0].annotation['modelseeds'] = ['one', 'two', 'three']

In [173]:
cobra.io.write_sbml_model(model_cobra, '../../../../data/sbml/ec_core/e_coli_core_test.xml')

In [19]:
model_kbase2 = kbase.get_from_ws('GCF_000005845.2.RAST_test_edit.mdl', 12218)

In [17]:
model_kbase = kbase.get_from_ws('e_coli_core.kb', 12218)

ERROR:cobrakbase.kbaseapi:Token validation failed: Login failed! Server responded with code 401 Unauthorized


In [3]:
from cobrakbase.core.kbasefba.fbamodel_protein import ModelReactionProtein

In [62]:
mrp_list = [ModelReactionProtein.from_json(o) for o in model_kbase2.reactions.rxn09272_c0.modelReactionProteins]
print(mrp_list)
gpr = " or ".join([f'({str(o)})' for o in mrp_list])
print(gpr)
print(model_kbase2.reactions.rxn09272_c0.gene_reaction_rule)

[b4151 and b4152 and b0722 and b0721, b0723, b4154, b0724, b4153]
(b4151 and b4152 and b0722 and b0721) or (b0723) or (b4154) or (b0724) or (b4153)
(b4151 and b4152 and b0722 and b0721) or b0723 or b4154 or b0724 or b4153


In [127]:
for o in mrp_list:
    o

In [128]:
o.get_data()

{'note': '',
 'source': '',
 'modelReactionProteinSubunits': [{'role': 'tftr00060',
   'note': '',
   'triggering': 1,
   'optionalSubunit': 0,
   'feature_refs': ['~/genome/features/id/b4153']}],
 'complex_ref': '~/template/complexes/name/tcpx00269'}

In [65]:
from modelseedpy.core.msmodel import get_set_set
gpr_sets = get_set_set("(b4151 and b4152 and b0722 and b0721) or b0723 or b4154 or b0724 or b4153")

In [ ]:
{'feature_refs': ['~/genome/features/id/b4151'],
             'note': '',
   'optionalSubunit': 0,
   'role': 'ftr07617',
   'triggering': 1}

In [79]:
template = kbase.get_from_ws(model_kbase2.template_ref)
genome  = kbase.get_from_ws(model_kbase2.genome_ref)

In [83]:
gene_mapping = {}
for gpr_cpx in gpr_sets:
    for gpr_cpx_subunit in gpr_cpx:
        if gpr_cpx_subunit not in gene_mapping:
            if gpr_cpx_subunit in genome.features:
                gene_mapping[gpr_cpx_subunit] = genome.features.get_by_id(gpr_cpx_subunit)
            else:
                gene_mapping[gpr_cpx_subunit] = None

In [89]:
template.reactions.rxn09272_c

Reaction identifier,rxn09272_c
Name,"fumarate reductase complex (i.e. FRD, involved in anaerobic respiration, repressed in aerobic..."
Memory address,0x7f10403ca310
Stoichiometry,cpd00036_c + cpd15560_c --> cpd00106_c + cpd15561_c Succinate + Ubiquinone-8 --> Fumarate + Ubiquinol-8
GPR,tcpx00267 or tcpx00268 or tcpx00269 or cpx00272 or cpx00108 or cpx00109 or tcpx00264 or cpx00107...
Lower bound,0
Upper bound,1000


In [87]:
for k in gene_mapping:
    print(gene_mapping[k].functions)

{'Succinate dehydrogenase iron-sulfur protein (EC 1.3.5.1)'}
{'Fumarate reductase subunit C'}
{'Fumarate reductase subunit D'}
{'Succinate dehydrogenase cytochrome b-556 subunit'}
{'Succinate dehydrogenase hydrophobic membrane anchor protein'}
{'Succinate dehydrogenase flavoprotein subunit (EC 1.3.5.1)'}
{'Fumarate reductase iron-sulfur protein (EC 1.3.5.4)'}
{'Fumarate reductase flavoprotein subunit (EC 1.3.5.4)'}


In [75]:
data_mrp = []
for gpr_cpx in gpr_sets:
    data_cpx = {
        'complex_ref': '~/template/complexes/name/cpx00108', #match role in template cpx
        'modelReactionProteinSubunits': [],
        'note': '',
        'source': ''
    }
    for gpr_cpx_subunit in gpr_cpx:
        data_su = {
            'feature_refs': ['~/genome/features/id/' + gpr_cpx_subunit],
            'note': '',
            'optionalSubunit': 0,
            'role': 'ftr07617', #look genome {gpr_cpx_subunit} role in template
            'triggering': 1
        }
        data_cpx['modelReactionProteinSubunits'].append(data_su)
        print(gpr_cpx_subunit)
    data_mrp.append(data_cpx)
data_mrp

b0724
b4152
b4151
b0721
b0722
b0723
b4153
b4154


[{'complex_ref': '~/template/complexes/name/cpx00108',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b0724'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1}],
  'note': '',
  'source': ''},
 {'complex_ref': '~/template/complexes/name/cpx00108',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b4152'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b4151'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b0721'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b0722'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1}],
  'note': '',
  'source': ''},
 {'complex_ref': '~/template/complexes/name/cpx00108',
  'mode

In [68]:
model_kbase2.reactions.rxn09272_c0.modelReactionProteins[0]

{'complex_ref': '~/template/complexes/name/cpx00108',
 'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b4151'],
   'note': '',
   'optionalSubunit': 0,
   'role': 'ftr07617',
   'triggering': 1},
  {'feature_refs': ['~/genome/features/id/b4152'],
   'note': '',
   'optionalSubunit': 0,
   'role': 'ftr07616',
   'triggering': 1},
  {'feature_refs': ['~/genome/features/id/b0722'],
   'note': '',
   'optionalSubunit': 0,
   'role': 'ftr09157',
   'triggering': 1},
  {'feature_refs': ['~/genome/features/id/b0721'],
   'note': '',
   'optionalSubunit': 0,
   'role': 'ftr09156',
   'triggering': 1}],
 'note': '',
 'source': ''}

In [57]:
len(model_kbase2.reactions.rxn09272_c0.modelReactionProteins)

5

In [53]:
for rxn in model_kbase2.reactions:
    mrp_list = [ModelReactionProtein.from_json(o) for o in rxn.modelReactionProteins]
    gpr = " or ".join([str(o) for o in mrp_list])
    print(rxn.id, gpr)
    print(rxn.id, rxn.gene_reaction_rule)
    print()

rxn00265_c0 (b0615) or (b0616) or (b0617)
rxn00265_c0 b0615 or b0616 or b0617

rxn00549_c0 (b2930 or b3925) or (b4232)
rxn00549_c0 (b2930 or b3925) or b4232

rxn01123_c0 (b3526)
rxn01123_c0 b3526

rxn14427_c0 (b1224 or b1468) and (b1465 or b1227) and (b1466 or b1226) and (b1467 or b1225) or (b1466 or b1226) or (b1467 or b1225) or (b1465 or b1227) or (b1224 or b1468) or (b2204) and (b2203) and (b2202) and (b2205) or (b3365) and (b3366) or (b2206)
rxn14427_c0 ((b1468 or b1224) and (b1465 or b1227) and (b1466 or b1226) and (b1467 or b1225)) or (b1226 or b1466) or (b1467 or b1225) or (b1227 or b1465) or (b1468 or b1224) or (b2204 and b2203 and b2202 and b2205) or (b3365 and b3366) or b2206

rxn00441_c0 (b0726)
rxn00441_c0 b0726

rxn05561_c0 (b0621) or (b4123) or (b4138) or (b3528)
rxn05561_c0 b0621 or b4123 or b4138 or b3528

rxn05040_c0 (b3041)
rxn05040_c0 b3041

rxn05581_c0 (b3927)
rxn05581_c0 b3927

rxn00569_c0 (b3366) or (b3365)
rxn00569_c0 b3366 or b3365

rxn00545_c0 (b3916) or (b1723

AttributeError: 'Biomass' object has no attribute 'modelReactionProteins'

In [33]:
model_cobra.reactions.PYK.gene_reaction_rule

'b1854 or b1676'

In [42]:
model_cobra.reactions.PYK.gene_name_reaction_rule

'pykA or pykF or (cydA and cydB)'

In [41]:
model_cobra.reactions.PYK.gene_reaction_rule = 'b1854 or b1676 or (b0733 and b0734)'

In [38]:
model_cobra.reactions.CYTBD

Reaction identifier,CYTBD
Name,Cytochrome oxidase bd (ubiquinol-8: 2 protons)
Memory address,0x7f1047022c40
Stoichiometry,2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_e + q8_c 2.0 H+ + 0.5 O2 O2 + Ubiquinol-8 --> H2O H2O + 2.0 H+ + Ubiquinone-8
GPR,(b0978 and b0979) or (b0733 and b0734)
Lower bound,0.0
Upper bound,1000.0


In [ ]:
model_kbase.reactions.CYTBD_c0 

In [32]:
rxn = model_kbase.reactions.CYTBD_c0

In [33]:
rxn.modelReactionProteins

[{'complex_ref': '',
  'modelReactionProteinSubunits': [{'feature_refs': ['12218/341/5/features/id/b0733',
     '12218/341/5/features/id/b0734',
     '12218/341/5/features/id/b0978',
     '12218/341/5/features/id/b0979'],
    'note': 'Imported GPR',
    'optionalSubunit': 0,
    'role': '',
    'triggering': 0}],
  'note': 'Imported GPR',
  'source': ''}]

In [38]:
model_kbase2.reactions.rxn00549_c0.modelReactionProteins

[{'complex_ref': '~/template/complexes/name/cpx01517',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b2930',
     '~/genome/features/id/b3925'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr04824',
    'triggering': 1}],
  'note': '',
  'source': ''},
 {'complex_ref': '~/template/complexes/name/cpx01519',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b4232'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr06142',
    'triggering': 1}],
  'note': '',
  'source': ''}]

In [ ]:
rxn.mo

In [31]:
for rxn in model_kbase.reactions:
    if 'string_attributes' in dir(rxn):
        bigg_id = rxn.string_attributes['original_id'][2:]
        rxn_bigg = model_cobra.reactions.get_by_id(bigg_id)
        print(rxn.id, rxn.gene_reaction_rule)
        print(bigg_id, rxn_bigg.gene_reaction_rule)
        print()
        #print(rxn.string_attributes['original_id'][2:])
    else:
        pass

CYTBD_c0 b0733 or b0734 or b0978 or b0979
CYTBD (b0978 and b0979) or (b0733 and b0734)

rxn01333_c0 b2464 or b0008
TALA b2464 or b0008

O2t_c0 
O2t s0001

rxn00085_c0 b3212 or b3213
GLUSy b3212 and b3213

rxn00225_c0 b3115 or b1849 or b2296
ACKr b3115 or b2296 or b1849

GLNabc_c0 b0809 or b0811 or b0810
GLNabc b0811 and b0810 and b0809

rxn01476_c0 b0767
PGL b0767

FRUpts2_c0 b2416 or b2415 or b1818 or b1819 or b1817
FRUpts2 b1817 and b1818 and b1819 and b2415 and b2416

rxn00747_c0 b3919
TPI b3919

ETOHt2r_c0 
ETOHt2r 

GLCpts_c0 b2417 or b2416 or b2415 or b1621 or b1818 or b1819 or b1817 or b1101
GLCpts (b2417 and b1101 and b2415 and b2416) or (b1817 and b1818 and b1819 and b2415 and b2416) or (b2417 and b1621 and b2415 and b2416)

PIt2r_c0 b2987 or b3493
PIt2r b2987 or b3493

rxn20162_c0 b2976 or b4014
MALS b4014 or b2976

RPI_c0 b4090 or b2914
RPI b2914 or b4090

rxn00336_c0 b4015
ICL b4015

rxn00558_c0 b4025
PGI b4025

rxn01106_c0 b3612 or b4395 or b0755
PGM b3612 or b4395 or b075

In [92]:
from modelseedpy import MSBuilder, RastClient
rast = RastClient()

In [93]:
b = MSBuilder(genome, template)

In [95]:
model_test = b.build('test')

In [103]:
for r in model_test.reactions:
    print(r)
    print(r.gene_reaction_rule)

rxn00265_c0: cpd00137_c0 --> cpd00029_c0 + cpd00032_c0
b0616 or b0615 or b0617
rxn00549_c0: cpd00001_c0 + cpd00290_c0 --> cpd00009_c0 + cpd00072_c0
(b2930 or b3925) or b4232
rxn01123_c0: cpd00002_c0 + cpd00176_c0 --> cpd00008_c0 + cpd00067_c0 + cpd02711_c0
b3526
rxn14427_c0: 4 cpd00067_c0 + cpd00209_c0 + cpd18074_c0 --> cpd00001_c0 + 2 cpd00067_e0 + cpd00075_c0 + cpd18072_c0
b2206 or ((b1466 or b1226) and (b1467 or b1225) and (b1465 or b1227) and (b1468 or b1224)) or (b2203 and b2204 and b2205 and b2202) or (b1468 or b1224) or (b1467 or b1225) or (b1466 or b1226) or (b1465 or b1227) or (b3365 and b3366)
rxn00441_c0: cpd00024_c0 + cpd00056_c0 + cpd00067_c0 --> cpd00011_c0 + cpd03189_c0
b0726
rxn05561_c0: cpd00067_e0 + cpd00106_e0 <=> cpd00067_c0 + cpd00106_c0
b4138 or b3528 or b0621 or b4123
rxn05040_c0: cpd00171_c0 --> cpd00047_c0 + cpd00067_c0 + cpd11225_c0
b3041
rxn05581_c0: cpd00100_c0 <=> cpd00100_e0
b3927
rxn00569_c0: 2 cpd00001_c0 + 3 cpd00006_c0 + cpd00013_c0 <-- 3 cpd00005_c0 +

In [97]:
len(template.reactions)

335

In [109]:

def apply_genome(model, genome):
    for g in model.genes:
        if g.id not in genome.features:
            print(g)
            
    model.genome = genome
    pass

apply_genome(model_test, genome)

In [111]:
rxn = model_test.reactions[0]

In [125]:
from cobrakbase.core.kbasegenome.genome import KBaseGenome
if 'genome' in dir(model_test) and type(model_test.genome) == KBaseGenome:
    print('!')
else:
    print('!!')

!


In [116]:
type(model_test.genome) == cobrakbase.core.kbasegenome.genome.KBaseGenome

True

In [124]:
model_test.genome = genome

In [129]:
type(model_test)

cobra.core.model.Model

In [ ]:
ModelReactionProtein

In [130]:
kbase.save_object('test_model', 12218, 'KBaseFBA.FBAModel', model_test)

AttributeError: 'dict' object has no attribute 'endswith'

In [133]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
converter = CobraModelConverter(model_test, None, None)
#json, fbamodel = converter.build()

In [151]:
from cobrakbase.core.kbasefba.fbamodel_metabolite import ModelCompound
from cobrakbase.core.kbasefba.fbamodel_reaction import ModelReaction, get_for_rev_flux_from_bounds
from modelseedpy.core.msmodel import get_direction_from_constraints
model_compounds = {}
for m in model_test.metabolites:
    model_compound = ModelCompound(m.id, m.formula, m.name, m.charge, m.compartment)
    if model_compound.id not in model_compounds:
        model_compounds[model_compound.id] = model_compound

In [162]:
model_compounds['cpd00001_c0']._to_json()

{'id': 'cpd00001_c0',
 'name': 'H2O_c0',
 'charge': 0,
 'formula': 'H2O',
 'compound_ref': '~/template/compounds/id/cpd00001',
 'modelcompartment_ref': '~/modelcompartments/id/c0'}

In [159]:
dict([(model_compounds[c.id], v) for (c, v) in r.metabolites.items()])

{<ModelCompound cpd01414_e0 at 0x7f1042079760>: -1.0,
 <ModelCompound cpd01414_c0 at 0x7f1042079790>: 1.0,
 <ModelCompound cpd00067_e0 at 0x7f109a47eb80>: -1.0,
 <ModelCompound cpd00067_c0 at 0x7f109a47eeb0>: 1.0}

In [160]:
model_reactions = []
for r in converter.metabolic_reactions:
    #direction = get_direction_from_constraints(r.lower_bound, r.upper_bound)
    #max_rev_flux, max_for_flux = get_for_rev_flux_from_bounds(r.lower_bound, r.upper_bound)
    model_reaction = ModelReaction(r.id, r.name, r.subsystem, r.lower_bound, r.upper_bound, imported_gpr=r.gene_reaction_rule)
    model_reaction.add_metabolites(dict([(model_compounds[c.id], v) for (c, v) in r.metabolites.items()]))
    print(model_reaction)

rxn00265_c0: cpd00137_c0 --> cpd00029_c0 + cpd00032_c0
rxn00549_c0: cpd00001_c0 + cpd00290_c0 --> cpd00009_c0 + cpd00072_c0
rxn01123_c0: cpd00002_c0 + cpd00176_c0 --> cpd00008_c0 + cpd00067_c0 + cpd02711_c0
rxn14427_c0: 4 cpd00067_c0 + cpd00209_c0 + cpd18074_c0 --> cpd00001_c0 + 2 cpd00067_e0 + cpd00075_c0 + cpd18072_c0
rxn00441_c0: cpd00024_c0 + cpd00056_c0 + cpd00067_c0 --> cpd00011_c0 + cpd03189_c0
rxn05561_c0: cpd00067_e0 + cpd00106_e0 <=> cpd00067_c0 + cpd00106_c0
rxn05040_c0: cpd00171_c0 --> cpd00047_c0 + cpd00067_c0 + cpd11225_c0
rxn05581_c0: cpd00100_c0 <=> cpd00100_e0
rxn00569_c0: 2 cpd00001_c0 + 3 cpd00006_c0 + cpd00013_c0 <-- 3 cpd00005_c0 + 5 cpd00067_c0 + cpd00075_c0
rxn00545_c0: cpd00002_c0 + cpd00072_c0 --> cpd00008_c0 + cpd00067_c0 + cpd00290_c0
rxn00874_c0: cpd00022_c0 + cpd00120_c0 <=> cpd00010_c0 + cpd03123_c0
rxn01333_c0: cpd00102_c0 + cpd00238_c0 <=> cpd00072_c0 + cpd00236_c0
rxn00224_c0: cpd01476_c0 + cpd10515_c0 <=> cpd00028_c0 + 2 cpd00067_c0
rxn00256_c0: cpd000

In [161]:
model_reaction._to_json()

{'id': 'rxn05730_c0',
 'name': 'Tetrathionate transport in via proton symport_c0',
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 1000.0,
 'aliases': [],
 'dblinks': {},
 'edits': {},
 'gapfill_data': {},
 'modelReactionProteins': [],
 'modelReactionReagents': [{'modelcompound_ref': '~/modelcompounds/id/cpd01414_e0',
   'coefficient': -1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd01414_c0', 'coefficient': 1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd00067_e0',
   'coefficient': -1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0',
   'coefficient': 1.0}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'probability': None,
 'protons': None,
 'reaction_ref': '~/template/reactions/id/rxn05730_c',
 'imported_gpr': ''}